In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [3]:
from tensorflow.keras.datasets import mnist, fashion_mnist

In [4]:
from tensorflow.keras.utils import to_categorical

### 讀取資料並整理

In [5]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [6]:
x_train.shape

(60000, 28, 28)

In [7]:
x_train = x_train.reshape(60000, 28, 28, 1)/225
x_test = x_test.reshape(10000, 28, 28, 1)/225

In [8]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 建立神經網路層

In [9]:
Conv_layer=[Conv2D(16, (3,3), padding='same', input_shape=(28,28,1)),
            Activation('relu'), 
            MaxPooling2D(pool_size=(2,2)),
                   
            Conv2D(32, (3,3), padding='same'),
            Activation('relu'),
            MaxPooling2D(pool_size=(2,2)),
                   
            Conv2D(64, (3,3), padding='same'),
            Activation('relu'),
            MaxPooling2D(pool_size=(2,2)),
                   
            Conv2D(128, (3,3), padding='same'),
            Activation('relu'),
            MaxPooling2D(pool_size=(2,2))]
fla_layer=[Flatten(),
           Dense(35), Activation('relu'),
           Dense(78), Activation('relu'),
           Dense(10), Activation('softmax')]

In [10]:
model=Sequential(Conv_layer + fla_layer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          1

In [11]:
model.compile(loss='mse', optimizer=SGD(lr=0.87),metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, batch_size=100, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 43s 719us/sample - loss: 0.0656 - accuracy: 0.4632
Epoch 2/10
60000/60000 [==============================] - 43s 714us/sample - loss: 0.0322 - accuracy: 0.7702
Epoch 3/10
60000/60000 [==============================] - 38s 633us/sample - loss: 0.0252 - accuracy: 0.8225
Epoch 4/10
60000/60000 [==============================] - 38s 627us/sample - loss: 0.0216 - accuracy: 0.8493
Epoch 5/10
60000/60000 [==============================] - 41s 683us/sample - loss: 0.0195 - accuracy: 0.8643
Epoch 6/10
60000/60000 [==============================] - 39s 651us/sample - loss: 0.0182 - accuracy: 0.8750
Epoch 7/10
60000/60000 [==============================] - 39s 652us/sample - loss: 0.0172 - accuracy: 0.8816
Epoch 8/10
60000/60000 [==============================] - 42s 694us/sample - loss: 0.0164 - accuracy: 0.8880
Epoch 9/10
60000/60000 [==============================] - 39s 647us/sample - loss: 0.0156 - accuracy: 0.8

### 保留convolutional layers，做新的神經網路

In [13]:
mnist_fla_layer=[Flatten(),
                 Dense(53), Activation('relu'),
                 Dense(10), Activation('softmax')]

In [14]:
mnist_model=Sequential(Conv_layer + mnist_fla_layer)

In [15]:
mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)         

### 將借來的神經網路冷凍起來

In [16]:
for layer in Conv_layer:
    layer.trainable = False

In [17]:
mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)         

In [18]:
(new_x_train, new_y_train), (new_x_test, new_y_test) = mnist.load_data()

In [19]:
new_x_train = new_x_train.reshape(60000, 28, 28, 1)/225
new_x_test = new_x_test.reshape(10000, 28, 28, 1)/225

In [20]:
new_y_train = to_categorical(new_y_train, 10)
new_y_test = to_categorical(new_y_test, 10)

In [21]:
mnist_model.compile(loss='mse', optimizer=SGD(lr=0.059), metrics=['accuracy'])

In [22]:
mnist_model.fit(new_x_train, new_y_train, batch_size=128, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.0865 - accuracy: 0.2368
Epoch 2/10
60000/60000 [==============================] - 11s 190us/sample - loss: 0.0689 - accuracy: 0.4961
Epoch 3/10
60000/60000 [==============================] - 12s 198us/sample - loss: 0.0556 - accuracy: 0.6294
Epoch 4/10
60000/60000 [==============================] - 12s 192us/sample - loss: 0.0472 - accuracy: 0.6923
Epoch 5/10
60000/60000 [==============================] - 11s 191us/sample - loss: 0.0418 - accuracy: 0.7280
Epoch 6/10
60000/60000 [==============================] - 12s 193us/sample - loss: 0.0379 - accuracy: 0.7506
Epoch 7/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0342 - accuracy: 0.7816
Epoch 8/10
60000/60000 [==============================] - 12s 200us/sample - loss: 0.0308 - accuracy: 0.8116
Epoch 9/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.0278 - accuracy: 0.8

### 查看預測結果

In [23]:
score = mnist_model.evaluate(new_x_test, new_y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [24]:
loss, acc = score

In [25]:
print('測試資料正確率', acc)

測試資料正確率 0.8606
